Decision Tree 의 경계면을 살펴보기 위하여 인공데이터인 two moon dataset 을 분류하는 문제를 풀어봅니다.

In [1]:
from bokeh.plotting import output_notebook, show
from decision_tree_utils import scatterplot_2class
from decision_tree_utils import draw_activate_image
from soydata.data.classification import make_moons

import numpy as np
np.set_printoptions(precision=5, suppress=True)

import warnings
warnings.filterwarnings('ignore')

output_notebook()

X, labels = make_moons(n_samples=500, xy_ratio=2.0, x_gap=-0.2, y_gap=-0.15, noise=0.1, seed=0)
p = scatterplot_2class(X, labels, height=400, width=400)
show(p)

Loading BokehJS ...

Decision tree 도 leaf node 의 (nega, posi) samples 의 개수로부터 확률을 계산할 수 있습니다. 이를 support vector machine 의 decision score 처럼 [-1, 1] 사이의 값으로 만들기 위하여 `posi. prob. - nega. prob.` 를 `score` 로 계산합니다. 그리고 모델의 정확도도 `accuracy` 에 저장합니다.

In [2]:
from sklearn.tree import DecisionTreeClassifier

def prepare_elements(model, X, labels):
    score = model.predict_proba(X)
    score = score[:,1] - score[:,0]
    pred = model.predict(X)    
    accuracy = (pred == labels).sum() / labels.shape[0]
    return score, accuracy

기본 하이퍼 패러매터로 의사결정나무를 학습한 뒤, score 를 기반으로 영역별 예측 레이블을 확인합니다. max depth = None 으로 설정되어있기 때문에 과적합되어 정확도가 1입니다. 의사결정나무는 xi = xj, yi != yj 가 아닌 이상 과적합을 하면 반드시 정확도가 100% 가 될 수 있습니다.

In [3]:
dt = DecisionTreeClassifier()
dt.fit(X, labels)

score, accuracy = prepare_elements(dt, X, labels)
title = f'Decision Tree. accuracy={accuracy:.4}'
p = draw_activate_image(dt, X, use_score=True, resolution=100, title=title, height=400, width=400)
p = scatterplot_2class(X, labels, score=score, p=p)
show(p)

과적합을 방지하기 위하여 max depth, min sample leaf 등의 하이퍼 패러매터를 설정합니다. 아래의 예시에서는 max depth 만 다르게 설정하였습니다. depth 가 작으면 under fitting 된 모델이 학습됨을 확인할 수 있습니다.

In [4]:
from bokeh.layouts import gridplot
from bokeh.io import save

figures = []
for depth in reversed([3, 5, 7, 10]):
    # train model
    dt = DecisionTreeClassifier(
        max_depth = depth,
        max_features = None,
        min_samples_leaf = 3,
        min_samples_split = 6,
        min_impurity_decrease = 0,
        min_impurity_split = None,
        criterion = 'gini' # ['gini', 'entropy']
    )
    dt.fit(X, labels)

    score, accuracy = prepare_elements(dt, X, labels)
    title = f'Decision Tree (depth={depth}). accuracy={accuracy:.4}'
    p = draw_activate_image(dt, X, use_score=True, resolution=100, title=title, height=400, width=400)
    p = scatterplot_2class(X, labels, score=score, p=p)
    figures.append(p)

gp = gridplot([figures[:2], figures[2:]])
show(gp)
# _ = save(gp, './figures/decision_tree_various_depth.html')

feature importances 는 의사결정나무가 학습하는 과정에서 각 변수 별 information gain 의 합을 누적한 값입니다.

In [5]:
# Sum of information gain by feature
dt.feature_importances_

array([0.41415, 0.58585])

`decision_path()` 함수를 이용하면 `X` 의 의사결정과정의 decision path 를 계산해줍니다. 그런데 `paths` 의 형식이 Compressed Sparse Raw (CSR) 입니다. 이는 sparse matrix 로, 우선 numpy.ndarray 로 변환하여 살펴봅니다.

In [6]:
paths = dt.decision_path(X)
paths

<500x11 sparse matrix of type '<class 'numpy.int64'>'
	with 1940 stored elements in Compressed Sparse Row format>

Sparse matrix 에 `todense()` 함수를 실행하면 numpy.ndarray 로 형식이 바뀝니다 그 중 X[0] 의 decision path 에 대해서만 확인합니다. 의사결정나무의 마디는 발생순서에 따라 0, 1, 2, ... 순서로 번호를 부여받습니다. X[0] 이 지나간 마디들은 1 로 표시된 마디들입니다.

In [7]:
path = paths.todense()[0]
print(path)

path = np.array(path)[0]
print(path)

[[1 0 0 0 0 0 1 1 1 0 0]]
[1 0 0 0 0 0 1 1 1 0 0]


의사결정나무의 feature, threshold, split 이후의 연결된 마디는 각각 다음의 attributes 에 나뉘어 저장되어 있습니다. Decision rules 은 < 입니다. X[i,j] 의 값이 threshold 보다 작으면 left, 크면 right 입니다.

In [8]:
# 의사결정에 이용한 변수
print(dt.tree_.feature)

# 의사결정에 이용한 변수의 threshold
print(dt.tree_.threshold)

# x[i,j] 가 threshold 보다 작을 때 이동하는 마디의 번호. leaves 이면 -1
print(dt.tree_.children_left)

# x[i,j] 가 threshold 보다 클 때 이동하는 마디의 번호. leaves 이면 -1
print(dt.tree_.children_right)

[ 1  0 -2  1 -2 -2  0  1 -2 -2 -2]
[-0.13932 -0.42641 -2.      -0.14526 -2.      -2.       1.43287  0.91485
 -2.      -2.      -2.     ]
[ 1  2 -1  4 -1 -1  7  8 -1 -1 -1]
[ 6  3 -1  5 -1 -1 10  9 -1 -1 -1]


각 마디마다 클래스 별 데이터가 몇 개씩 위치하는지에 대한 정보는 `value` 에 저장되어 있습니다. 첫번째 마디는 root 이기 때문에 (250, 250) 개의 데이터가 포함되어 있습니다.

In [9]:
print(dt.tree_.value)

[[[250. 250.]]

 [[  5. 129.]]

 [[  3.   0.]]

 [[  2. 129.]]

 [[  1. 127.]]

 [[  1.   2.]]

 [[245. 121.]]

 [[245.  64.]]

 [[ 85.  52.]]

 [[160.  12.]]

 [[  0.  57.]]]


이 정보들을 이용하면 X[0] 이 지나간 궤적에 대하여 해석가능하게 출력할 수 있습니다.

In [10]:
def print_path(path, dt, xi):
    feature = dt.tree_.feature
    threshold = dt.tree_.threshold
    left = dt.tree_.children_left
    right = dt.tree_.children_right
    value = dt.tree_.value

    indices = np.where(path > 0)[0]
    elements = [(i, feature[i], threshold[i], left[i], right[i]) for i in indices]

    print(f'Input : {xi}\n')
    for node, var, t, l, r in elements:
        if l == -1 or r == -1:
            neg, pos = value[node][0].astype(np.int)
            label = 0 if neg > pos else 1
            print(f'Leaf {node}: ({neg}, {pos}), label={label}')
        else:
            dest = r if xi[var] > t else l
            direction = '>' if xi[var] > t else '<'
            print(f'Node {node} -> {dest} (x{var} {direction} {t})')

print_path(path, dt, X[0])

Input : [ 1.37641 -0.06997]

Node 0 -> 6 (x1 > -0.1393222063779831)
Node 6 -> 7 (x0 < 1.4328681826591492)
Node 7 -> 8 (x1 < 0.9148471355438232)
Leaf 8: (85, 52), label=0


만약 각 데이터의 leaf node 의 인덱스만 가져오고 싶다면 sparse matrix 를 분해해야 합니다. 이에 대한 자세한 설명은 아래의 블로그를 참고하시기 바랍니다.

https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/

In [11]:
print(type(paths))
print(paths.indices.shape)
print(paths.indptr.shape)

<class 'scipy.sparse.csr.csr_matrix'>
(1940,)
(501,)


In [12]:
print(paths.indices[:14])
print(paths.indptr[:4])

[0 6 7 8 0 6 7 8 0 1 3 4 0 6]
[ 0  4  8 12]


아래의 코드는 임의의 의사결정나무에 대하여 사용가능합니다. 모든 데이터에 대하여 leaf node 의 인덱스를 가져올 수 있습니다.

In [13]:
leaves = paths.indices[paths.indptr[1:] - 1]
leaves.shape

(500,)

In [14]:
leaves[:3]

array([8, 8, 4], dtype=int32)